In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.isnull().sum()
test = test_data

In [4]:
X = train_data.drop("Survived",axis=1)

y = train_data.Survived


In [5]:
X.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
cato = []
numo = []
for i in X.columns:
    if X[i].nunique()<=10 and X[i].dtypes=="object":
        cato.append(i)
for i in X.columns:
    if X[i].dtypes=="int64":
        numo.append(i)
        

In [7]:
numo

['PassengerId', 'Pclass', 'SibSp', 'Parch']

In [8]:
X.Ticket
cato

['Sex', 'Embarked']

In [9]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_int_X = pd.DataFrame(si.fit_transform(X[numo]))
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputed_cato_X = pd.DataFrame(si.fit_transform(X[cato]))

si = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_int_test = pd.DataFrame(si.fit_transform(test[numo]))
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputed_cato_test = pd.DataFrame(si.fit_transform(test[cato]))

In [10]:
imputed_int_X.columns = numo
imputed_cato_X.columns = cato
imputed_int_test.columns = numo
imputed_cato_test.columns = cato


In [11]:
imputed_cato_X.nunique()

Sex         2
Embarked    3
dtype: int64

In [12]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
imputed_cato_X= pd.DataFrame(ordinal_encoder.fit_transform(imputed_cato_X))


imputed_cato_test= pd.DataFrame(ordinal_encoder.fit_transform(imputed_cato_test))
imputed_cato_X

,0,1
0,1.0,2.0
1,0.0,0.0
2,0.0,2.0
3,0.0,2.0
4,1.0,2.0
...,...,...
886,1.0,2.0
887,0.0,2.0
888,0.0,2.0
889,1.0,0.0


In [13]:



final_X = pd.concat([imputed_int_X,imputed_cato_X],axis=1)
final_test = pd.concat([imputed_int_test,imputed_cato_test],axis=1)

In [14]:
X =final_X

In [15]:
final_test

,PassengerId,Pclass,SibSp,Parch,0,1
0,892.0,3.0,0.0,0.0,1.0,1.0
1,893.0,3.0,1.0,0.0,0.0,2.0
2,894.0,2.0,0.0,0.0,1.0,1.0
3,895.0,3.0,0.0,0.0,1.0,2.0
4,896.0,3.0,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...
413,1305.0,3.0,0.0,0.0,1.0,2.0
414,1306.0,1.0,0.0,0.0,0.0,0.0
415,1307.0,3.0,0.0,0.0,1.0,2.0
416,1308.0,3.0,0.0,0.0,1.0,2.0


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [17]:
X_train.dtypes

PassengerId    float64
Pclass         float64
SibSp          float64
Parch          float64
0              float64
1              float64
dtype: object

In [18]:
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=500, learning_rate=0.05)
my_model.fit(X_train, y_train,early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=True)


[0]	validation_0-rmse:0.47538
[1]	validation_0-rmse:0.46510
[2]	validation_0-rmse:0.45552
[3]	validation_0-rmse:0.44676
[4]	validation_0-rmse:0.43890
[5]	validation_0-rmse:0.43173
[6]	validation_0-rmse:0.42505
[7]	validation_0-rmse:0.41903
[8]	validation_0-rmse:0.41341
[9]	validation_0-rmse:0.40846
[10]	validation_0-rmse:0.40409
[11]	validation_0-rmse:0.39998
[12]	validation_0-rmse:0.39605
[13]	validation_0-rmse:0.39301
[14]	validation_0-rmse:0.39071
[15]	validation_0-rmse:0.38802
[16]	validation_0-rmse:0.38626
[17]	validation_0-rmse:0.38419
[18]	validation_0-rmse:0.38247
[19]	validation_0-rmse:0.38091
[20]	validation_0-rmse:0.37931
[21]	validation_0-rmse:0.37807
[22]	validation_0-rmse:0.37640
[23]	validation_0-rmse:0.37537
[24]	validation_0-rmse:0.37422
[25]	validation_0-rmse:0.37342
[26]	validation_0-rmse:0.37309
[27]	validation_0-rmse:0.37252
[28]	validation_0-rmse:0.37180
[29]	validation_0-rmse:0.37133
[30]	validation_0-rmse:0.37107
[31]	validation_0-rmse:0.37072
[32]	validation_0-

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[44]	validation_0-rmse:0.36930
[45]	validation_0-rmse:0.36922
[46]	validation_0-rmse:0.36928
[47]	validation_0-rmse:0.36939
[48]	validation_0-rmse:0.36945
[49]	validation_0-rmse:0.36921
[50]	validation_0-rmse:0.36911
[51]	validation_0-rmse:0.36903
[52]	validation_0-rmse:0.36899
[53]	validation_0-rmse:0.36884
[54]	validation_0-rmse:0.36884
[55]	validation_0-rmse:0.36874
[56]	validation_0-rmse:0.36886
[57]	validation_0-rmse:0.36878
[58]	validation_0-rmse:0.36893
[59]	validation_0-rmse:0.36898


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [19]:
my_model.best_iteration

55

In [20]:

my_model = XGBRegressor(n_estimators=25, learning_rate=0.05)
my_model.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=25, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [21]:
y_pred_probs  = my_model.predict(final_test)

In [22]:
threshold = 0.5
y_pred_binary = (y_pred_probs >= threshold).astype(int)

In [23]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [24]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred_binary})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
